In [2]:
desktop_path = '/Users/shaanpakala/Desktop/'
root_path = f'{desktop_path}CS_STUFF/Python/NLP/Sentiment_Analysis/'
data_path = f'{root_path}data/'
saved_model_path = f'{root_path}saved_models/'
train_vect_path = f'{root_path}train_vectors/'

In [4]:
files = [f'{data_path}yellowpages.txt', f'{data_path}yelp_reviews.txt', f'{data_path}kaggle_twitter.txt', 
         f'{data_path}kaggle_reddit.txt']

In [1]:
def getFileText():
    text = []
    for i in range(len(files)):

        file = files[i]

        f = open(file, encoding="utf8")
        t = f.read().split('\n - \n - \n')
        f.close()

        if (i == 2): t = t[:20000]

        elif (i == 3): t = t[:20000]

        text+=t
    
        
    return text[:-2]

In [2]:
adverbs = ['very', 'too', 'really', 'extremely']

In [4]:
randomchars = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '&', ';', 'yyy', 'ooo', '\\', '.', '!', ':', '/',
              '+', '-', '@', '*', '(', ')', '#', 'hhh', ',', '$', '😉', 'ó', 'é', 'lll', '%', '_','uuu','ooo','sss',
               'ppp', '✌', '🤡', 'haha', 'lol', 'lmao', 'eee', 'ė', 'į', "'"]

In [8]:
stopWords = ['i', 'and', 'at', 'the', 'yet', 'so', 'because', 'on', 'of', 'to', 'as',' in', 'his', 'her', 'she', 'him',
            'them', 'they', 'it', 'hers', 'their', 'its', 'theirs', 'with', 'said', 'for', 'after', 'will', 'that',
            'about', 'who', 'by', 'all', 'where', 'over', 'year', 'years', 'continue', 'two', 'three', 'four', 'five',
            'six', 'seven', 'eight', 'nine', 'ten', 'other', 'into', 'a', 'an', 'in', 'my', 'is', 'but', 'he', 'me',
            'from', 'am', 'we', 'ok', 'okay', 'you', 'us', 'have', 'are', 'your', 'own', 'had', 'or', 'this', 'has',
            'hella', 'yo', 'est', "he's", "she's", "we're", "they're", "we'll", "they'll", "i'm", "that's", "who's",
            "what", "when", "how", "it's", "i'll", "she'll", "he'll", "it'll", "that'll", "who'll", "might've",
            "would've", "could've", "should've", "here's", "what's", "i've", "it'd", "we'd", "they'd"]

In [9]:
negationWords = ['not', 'no']

In [11]:
negContractions = {"shouldn't":'should not', "wouldn't":"would not", "cannot":"can not", "can't":"can not",
               "aren't":"are not", "ain't":"is not", "don't":"do not", "couldn't":"could not", "haven't":"have not",
               "needn't":"need not", "didn't":"did not", "wasn't":"was not", "weren't":"were not", "won't":"will not",
               "hasn't":"has not", "doesn't":"does not", "isn't":"is not",
                   
                "shouldnt":'should not', "wouldnt":"would not", "cant":"can not",
               "arent":"are not", "aint":"is not", "dont":"do not", "couldnt":"could not", "havent":"have not",
               "neednt":"need not", "didnt":"did not", "wasnt":"was not", "werent":"were not", "wont":"will not",
               "hasnt":"has not", "doesnt":"does not", "isnt":"is not"}

In [1]:
def trimWord(word):
    w = word
    
    if ((w[len(w)-1] == ",")or(w[len(w)-1] == ".")or(w[len(w)-1] == "?")or(w[len(w)-1] == "%")
        or(w[len(w)-1] == "!")or(w[len(w)-1] == "’")or(w[len(w)-1] == ":")or(w[len(w)-1] == ")")or(w[len(w)-1] == ";")
        or(w[len(w)-1]=="\"")or(w[len(w)-1]=="-")or(w[len(w)-1]=="'")or(w[len(w)-1]=="”")or(w[len(w)-1]=="‘")
        or(w[len(w)-1]=="]")or(w[len(w)-1]==">")):
        w = w[:len(w)-1]          
    elif((w[0]=="'")or(w[0]=="’")or(w[0]=="\"")or(w[0]=="-")or(w[0]=="“")or(w[0]=="(")or(w[0]=="‘")
        or(w[0]=="[")or(w[0]=="”")):
        w = w[1:]          
    elif ((w[len(w)-2:len(w)] == "’s")or(w[len(w)-2:len(w)] == "\'s")):
        w = w[:len(w)-2]
    else:
        return w
    
    
    return(trimWord(w))

In [3]:
def hasVowel(word):
    vowels = 'aeiouy'
    
    for i in word:
        if (vowels.find(i) != -1): return True
        
    return False

In [3]:
def goodWord(w):
    
    if (len(w) > 15): return False
    
    if (not hasVowel(w)): return False
    
    if (w in stopWords): return False
    
    for c in randomchars:
        if (w.find(c)!=-1): return False
    
    return True

In [4]:
str_line = '_____________________________________________________________________________________________________________________'

In [7]:
def sentiment_to_num(sentiment):
    if(sentiment == 'negative'): return -1
    elif(sentiment == 'neutral'): return 0
    elif(sentiment == 'positive'): return 1
    else: return None

In [1]:
def num_to_sentiment(num):
    if(num == -1): return 'negative'
    elif(num == 0): return 'neutral'
    elif(num == 1): return 'positive'
    else: return None

In [1]:
def vectorize(text, w_d):
    vector = [0 for x in range(len(w_d))]
    
    words = text.split()
    textLen = len(words)

    for i in range(textLen):
        try: word = trimWord(words[i]).lower()
        except: continue
        
        if (not goodWord(word)): continue

        if (word in negContractions): word = 'not'

        if (word == 'not'): 
            try: 
                nextWord = trimWord(words[i+1]).lower()
                if (not goodWord(nextWord)): continue
                word = (f'{word}_{nextWord}')

            except: continue

        elif (word in adverbs): 
            try: 
                nextWord = trimWord(words[i+1]).lower()
                if (not goodWord(nextWord)): continue
                word = (f'{word}_{nextWord}')

            except: continue

        try: vector[word_dict.index(word)]+=1
        except: continue
    
    length = sum(vector)
    
    try: new_vector = [(x / length) for x in vector]
    except: return 0
    
    return new_vector

In [4]:
def classify_from_sentence(text, clf, word_dict):
    
    vector = vectorize(text, word_dict)
    vector_np = np.array(vector)
    vector_torch = torch.tensor(vector_np)
    single_vect_torch = vector_torch.unsqueeze(0)
    prediction = int(classifier.predict(single_vect_torch))
    
    if (prediction == -1): sentiment = 'negative'
    elif (prediction == 0): sentiment = 'neutral'
    elif (prediction == 1): sentiment = 'positive'
    else: sentiment = None
    
    return sentiment

In [1]:
def sentiment(s, classifier, word_dict):
    
    try: class_ = classify_from_sentence(s, classifier, word_dict)        
    except: return 'Error'
        
    party = s.split()[len(s.split())-1]
    
    if ((party == 'r') or (party == 'd')): return [class_, party]
    
    else: return class_